In [5]:
%matplotlib notebook

In [6]:
#Dependencies and Setup
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
import json

#Import API key
from api_key import api_key

#Output File (CSV)
data1="NPD_Data_2018_2019_V2.csv"
data2="NPD_Data_2018_2019_V3.csv"

#Read CSV files
v2=pd.read_csv(data1)
v3=pd.read_csv(data2)

In [7]:
v2.head()

,Month,Retailer,StoreType,Category,Units
0,2018 Apr,AAFES,B&M,DAYPACK,0
1,2018 Apr,AAFES,B&M,JACKET/VEST,60
2,2018 Apr,AAFES,B&M,PANT/CAPRI,7401
3,2018 Apr,AAFES,ECOMMERCE,SLEEPING BAG,0
4,2018 Apr,AAFES,OTHER,PANT/CAPRI,-15


In [8]:
v3.head()

,Year,Month,Retailer,StoreType,State,Region,Category,TY AUR,Units
0,2019,Feb,AAFES,B&M,AE,OTHER,JACKET/VEST,NaN,0
1,2019,Feb,AAFES,B&M,AE,OTHER,PANT/CAPRI,NaN,14
2,2019,Feb,AAFES,B&M,AK,NORTHWEST,JACKET/VEST,NaN,0
3,2019,Feb,AAFES,B&M,AK,NORTHWEST,PANT/CAPRI,NaN,28
4,2019,Feb,AAFES,B&M,AL,SOUTHEAST,JACKET/VEST,NaN,2


How many retailers for each version of data

In [9]:
v2_retailers=v2["Retailer"].nunique()
v2_retailers

38

In [10]:
retailers2=v2["Retailer"].unique()
retailers2

array(['AAFES', 'ACADEMY SPORTS', 'BACKCOUNTRY', 'BLOOMINGDALES',
       'BOBS STORES', 'ChampaignSurplus', 'DICKS SPORTING GOODS',
       'DILLARDS', 'EASTERN MOUNTAIN SPORTS', 'FIELDANDSTREAM',
       'FINISH LINE', 'FOOT LOCKER, INC.', 'HIBBETT SPORTS', 'MACYS',
       'MARINECORPSEXCHANGE', 'MODELLS', 'MOOSEJAW', 'NEIMAN MARCUS',
       'NEXCOM', 'NORDSTROMPORTAL', 'OLYMPIA SPORTS',
       'PETER GLENN_GREEN MOUNTAIN 3P ', 'REI', 'ROCK CREEK',
       'RUNNINGSPECIALTYGROUP', 'SCHEELSSPORTS', 'VAILRESORTSRETAIL',
       'VON MAUR', 'ZAPPOS.COM', 'PACIFICSUNWEAR', 'JIMMYJAZZ',
       'SummitHut', 'HALFMOONOUTFITTERS', 'RAMSEYOUTDOOR', 'BassPro Domo',
       'CabelasDomo', 'AlsSportingGoods', 'Olympia'], dtype=object)

In [11]:
v3_retailers=v3["Retailer"].nunique()
v3_retailers

29

In [13]:
retailers3=v3["Retailer"].unique()
retailers3

array(['AAFES', 'ACADEMY SPORTS', 'BACKCOUNTRY', 'BLOOMINGDALES',
       'BOBS STORES', 'DICKS SPORTING GOODS', 'DILLARDS',
       'EASTERN MOUNTAIN SPORTS', 'FIELDANDSTREAM', 'FINISH LINE',
       'HIBBETT SPORTS', 'MACYS', 'MARINECORPSEXCHANGE', 'MODELLS',
       'MOOSEJAW', 'NEIMAN MARCUS', 'NEXCOM', 'NORDSTROMPORTAL',
       'OLYMPIA SPORTS', 'PACIFICSUNWEAR', 'REI', 'RUNNINGSPECIALTYGROUP',
       'SCHEELSSPORTS', 'VAILRESORTSRETAIL', 'VON MAUR', 'ZAPPOS.COM',
       'BassPro Domo', 'CabelasDomo', 'THENORTHFACEDTC'], dtype=object)

Drop retailers that aren't in both lists

In [23]:
drop_retailer = v2.set_index("Retailer")
drop_retailer.head()

,Month,StoreType,Category,Units
Retailer,,,,
AAFES,2018 Apr,B&M,DAYPACK,0
AAFES,2018 Apr,B&M,JACKET/VEST,60
AAFES,2018 Apr,B&M,PANT/CAPRI,7401
AAFES,2018 Apr,ECOMMERCE,SLEEPING BAG,0
AAFES,2018 Apr,OTHER,PANT/CAPRI,-15


In [25]:
drop2=drop_retailer.drop(["ChampaignSurplus","FOOT LOCKER, INC.", "PETER GLENN_GREEN MOUNTAIN 3P ", "ROCK CREEK", "JIMMYJAZZ", "SummitHut", "HALFMOONOUTFITTERS", "RAMSEYOUTDOOR", "AlsSportingGoods", "Olympia"])
drop2

,Month,StoreType,Category,Units
Retailer,,,,
AAFES,2018 Apr,B&M,DAYPACK,0
AAFES,2018 Apr,B&M,JACKET/VEST,60
AAFES,2018 Apr,B&M,PANT/CAPRI,7401
AAFES,2018 Apr,ECOMMERCE,SLEEPING BAG,0
AAFES,2018 Apr,OTHER,PANT/CAPRI,-15
...,...,...,...,...
ZAPPOS.COM,2021 Apr,OTHER,SLEEPING BAG,0
ZAPPOS.COM,2021 Apr,OTHER,TENT,0
ZAPPOS.COM,2021 Apr,OUTLET,DAYPACK,192


In [28]:
drop_retailer3 = v3.set_index("Retailer")
drop_retailer3

,Year,Month,StoreType,State,Region,Category,TY AUR,Units
Retailer,,,,,,,,
AAFES,2019,Feb,B&M,AE,OTHER,JACKET/VEST,NaN,0
AAFES,2019,Feb,B&M,AE,OTHER,PANT/CAPRI,NaN,14
AAFES,2019,Feb,B&M,AK,NORTHWEST,JACKET/VEST,NaN,0
AAFES,2019,Feb,B&M,AK,NORTHWEST,PANT/CAPRI,NaN,28
AAFES,2019,Feb,B&M,AL,SOUTHEAST,JACKET/VEST,NaN,2
...,...,...,...,...,...,...,...,...
ZAPPOS.COM,2020,Jan,ECOMMERCE,OTHER,OTHER,PANT/CAPRI,52.16,588
ZAPPOS.COM,2020,Jan,ECOMMERCE,OTHER,OTHER,TENT,NaN,0
ZAPPOS.COM,2020,Jan,OUTLET,OTHER,OTHER,DAYPACK,52.57,246


In [27]:
drop3=drop_retailer3.drop("THENORTHFACEDTC")
drop3.head()

,Year,Month,StoreType,State,Region,Category,TY AUR,Units
Retailer,,,,,,,,
AAFES,2019,Feb,B&M,AE,OTHER,JACKET/VEST,NaN,0
AAFES,2019,Feb,B&M,AE,OTHER,PANT/CAPRI,NaN,14
AAFES,2019,Feb,B&M,AK,NORTHWEST,JACKET/VEST,NaN,0
AAFES,2019,Feb,B&M,AK,NORTHWEST,PANT/CAPRI,NaN,28
AAFES,2019,Feb,B&M,AL,SOUTHEAST,JACKET/VEST,NaN,2


How many different categories for each version of data

In [12]:
v2_cat=v2["Category"].unique()
v2_cat

array(['DAYPACK', 'JACKET/VEST', 'PANT/CAPRI', 'SLEEPING BAG', 'TENT'],
      dtype=object)

In [14]:
v3_cat=v3["Category"].unique()
v3_cat

array(['JACKET/VEST', 'PANT/CAPRI', 'DAYPACK', 'SLEEPING BAG', 'TENT'],
      dtype=object)